In [1]:
import sys
sys.path.insert(0, '../')
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 9204.73it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 5122.65it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 3118011.96it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 9439.29it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4995.98it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 94513.35it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8113.42it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 5061.39it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 95527.60it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 8272.03it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 3997.12it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 92974.99it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
model = mz.models.ArcIIModel()
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 50
model.params['embedding_trainable'] = False
model.params['optimizer'] = 'adam'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to ArcIIModel.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             833700      text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 32)       4832        embeddin

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=50)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
pred_x, pred_y = predict_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))
len(pred_y)

2341

In [10]:
train_generator = mz.PairDataGenerator(train_pack_processed, num_dup=2, num_neg=1, batch_size=20)
len(train_generator)

102

In [11]:
history = model.fit_generator(train_generator, epochs=3, callbacks=[evaluate], workers=3, use_multiprocessing=True)

Epoch 1/3
102/102 [==============================] - 2s 16ms/step - loss: 0.6989
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.525158 - normalized_discounted_cumulative_gain@5(0):0.584905 - mean_average_precision(0):0.541072
Epoch 2/3
102/102 [==============================] - 2s 18ms/step - loss: 0.3547
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.491527 - normalized_discounted_cumulative_gain@5(0):0.564802 - mean_average_precision(0):0.522820
Epoch 3/3
102/102 [==============================] - 2s 21ms/step - loss: 0.1788
Validation: loss:nan - normalized_discounted_cumulative_gain@3(0):0.501231 - normalized_discounted_cumulative_gain@5(0):0.570541 - mean_average_precision(0):0.530823
